### Using models that were retrained on filtered training set

In [1]:
#Allows relative imports
import os
import sys
import pandas as pd 

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#imports 
from src.preprocessing import *
from src.models import *
from src.train_eval_helpers import *
from src.plots import *
import torch
import torch.nn as nn
import torch.nn.functional as F
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 300
import seaborn as sns
sns.set_style('darkgrid')

#checking gpu status
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))

RANGE = range(12,17)
TRAINDIR = '../TrainingData/'

Using : cuda


In [10]:
deepcat_none = load_models('../output/training_output/deepcat_NotScaled_filtered/', arch = 'deepcat')
deepcat_scaled = load_models('../output/training_output/deepcat_scaled_filtered/', arch = 'deepcat')
richie_minmax = load_models('../output/training_output/richie_minmax_filtered/', arch = 'richie')

Models & weights loaded
Models & weights loaded
Models & weights loaded


### McPAS-TCR

In [16]:
mcpas.columns

Index(['CDR3.alpha.aa', 'CDR3.beta.aa', 'Species', 'Category', 'Pathology',
       'Pathology.Mesh.ID', 'Additional.study.details',
       'Antigen.identification.method', 'Single.cell', 'NGS',
       'Antigen.protein', 'Protein.ID', 'Epitope.peptide', 'Epitope.ID', 'MHC',
       'Tissue', 'T.Cell.Type', 'T.cell.characteristics', 'CDR3.alpha.nt',
       'TRAV', 'TRAJ', 'TRBV', 'TRBD', 'TRBJ', 'Reconstructed.J.annotation',
       'CDR3.beta.nt', 'Mouse.strain', 'PubMed.ID', 'Remarks'],
      dtype='object')

In [38]:
mcpas = pd.read_csv('../NewData/McPAS-TCR.csv', header=0,usecols=['CDR3.beta.aa', 'Species','Category',
                                                                  'Pathology','MHC','T.Cell.Type'])
mcpas.dropna(inplace=True)
mcpas=mcpas.query('Species=="Human"&`T.Cell.Type`=="CD8"')
mcpas=mcpas.query('`CDR3.beta.aa`.str.startswith("C")')
mcpas_cancer= mcpas.query('Category=="Cancer"')
mcpas_not = mcpas.query('Category!="Cancer"')

In [ ]:
for x in mcpas_not_seq:
    print(x)

In [62]:
mcpas_cancer_seq = mcpas_cancer['CDR3.beta.aa'].values
mcpas_cancer_seq = [x.upper() for x in mcpas_cancer_seq]
mcpas_not_seq = mcpas_not['CDR3.beta.aa'].values
mcpas_not_seq = [x.upper() for x in mcpas_not_seq]

rich_mcpas_feats, rich_mcpas_labels = generate_features_labels(mcpas_cancer_seq, mcpas_not_seq, 
                                                               encoding = 'aa_atchley', scaling='minmax')
deep_mcpas_feats, deep_mcpas_labels = generate_features_labels(mcpas_cancer_seq, mcpas_not_seq, 
                                                               encoding = 'aaidx', scaling='minmax')


Getting data
Data device = cpu
Done loading, returning features and labels.
Getting data
Data device = cpu
Done loading, returning features and labels.


In [63]:
_, dc_accs, dc_aucs, dc_f1, dc_curves = test_eval(deepcat_scaled, deep_mcpas_feats, deep_mcpas_labels,range(12,17), nn.CrossEntropyLoss(), return_curve=True)
_, rich_accs, rich_aucs, rich_f1, rich_curves =  test_eval(richie_minmax,rich_mcpas_feats, rich_mcpas_labels ,range(12,17), nn.CrossEntropyLoss(), return_curve=True)
plot_roc_comparison(dc_curves, rich_curves, name='RETRAINED MODELS:McPas',save=None)#, keys=range(12,17), save=None)

### VDJdb

In [5]:
def parse_tsv(filepath):
    temp = pd.read_csv(filepath, sep='\t',header = 0, usecols=['CDR3','MHC class','Epitope','Epitope gene', 'Epitope species'])
    temp = temp.dropna()
    temp['len']= temp.apply(lambda x: len(x['CDR3']),axis=1)
    temp = temp.query('(CDR3.str.startswith("C") and CDR3.str.endswith("F"))')
    temp = temp.query('not CDR3.str.contains("\*")')
    temp = temp.query('`MHC class` != "MHCII"')
    temp = temp.query('len>=12 and len <=16')
    temp = temp.query('`Epitope gene`!="synthetic"')
    return temp.sort_values('Epitope gene')
def parse_csv(filepath):
    temp = pd.read_csv(filepath, header = 0, usecols=['epitope','cdr3_TRB'])
    temp = temp.dropna()
    temp['len']= temp.apply(lambda x: len(x['cdr3_TRB']),axis=1)
    temp = temp.query('(cdr3_TRB.str.startswith("C") and cdr3_TRB.str.endswith("F"))')
    temp = temp.query('not cdr3_TRB.str.contains("\*")')
    temp = temp.query('len>=12 and len <=16')
    return temp
epitope_map = {
    "ELAGIGILTV" : 1 #Melanoma MLANA
    ,"GILGFVFTL"  : 0 #Influenza A M gene
    ,"SLFNTVATLY" : 0 #HIV
    ,"RMFPNAPYL" : 1 #WT1
    ,"LLDFVRFMGV" : 1 #EBNA-B EBV but also linked to 
    ,"GLCTLVAML" : 1 #BMLF1 ebv
    ,"LLFGYPVYV" : 0 #HTLV-1 Tax
    ,"SLLMWITQV" : 1 #well known squamous cell NY-ESO
    ,"FLYALALLL" : 1 #EBV LMP2A, but linked to lymphoma
    ,"FLASKIGRLV" : 0 # PLA2G6 phospholipase
    ,"MLDLQPETT" : 1 #HPV E7
    ,"IMDQVPFSV" : 1 #PMEL melanosome....
    ,"YLLEMLWRL" : 1 #EBV lmp1 viral oncogene
    ,"KVLEYVIKV" : 1 #MAGE A1
    ,"RTLNAWVKV" : 0 #HIV gag1
    ,"KTWGQYWQV" : 1 #PMEL
    ,"YLNDHLEPWI" : 1 #BCL-2-l1
    ,"SLFNTVATL" : 0 #HIV Gag 1
    ,"CLLWSFQTSA" : 0 #tyr (ALBINISM)
    ,"ILKEPVHGV" : 0 #HIV1 POL
    ,"CLLGTYTQDV" : 0 #KANAMYCYN KANJ (ANTIBIO)
    ,"NLVPMVATV" :  0 #H CMV
    ,"KLQCVDLHV" : 0 # klk3
    ,"ELAGIGLTV" : 1 #MLANA
    ,"VLEETSVML" : 0 #CMV IE1
    ,"YVLDHLIVV" :  0 #HPV brlf1
    ,"FLGKIWPSHK" : 0 # HIV gag1
    ,"CLNEYHLFL" : 0 # human NDC1
    ,"AMFWSVPTV" : 0 #TKT phosphate glycolysis 
    ,"FLYNLLTRV" :  0 #SEC24A
    ,"KLMNIQQKL" : 1 #AKAP13
    ,"IILVAVPHV" : 0 #EXOC8
    ,"MLGEQLFPL" : 0 #PABPC1
    ,"YILEETSVM" : 0 # CMV IE1
    ,"CINGVCWTV" :  1 #NS3 Hep C??
    ,"KLSALGINAV" : 1 #NS3 Hep C??
    ,"LLWNGPMAV" :  1 #NS4B Hep C??
    ,"TLNAWVKVV" : 0 #HIV GAG1
    ,"KINAWIKVV" : 0 #HIV gag
    ,"SLYNTVATL" : 0 #HIV gag
    ,"CVNGSCFTV" : 0 #Influenza A
    ,"ARNLVPMVATVQGQN" : 0 #pp65
    ,"LSEFCRVLCCYVLEE" : 0 #CMV IE1
    ,"LLFGYAVYV" : 0 # HLTV tax
    ,"LLFGYPRYV" : 0 #HLTV tax
    ,"SLLMWITQC" : 1 #NY-ESO-1
    ,"LLFGKPVYV" : 0 #HLTV tax
    ,"LLFGPVYV" : 0 #HLTV tax
    ,"MLWGYLQYV" : 0 #HLTV tax
    ,"LGYGFVNYI" : 0 #TAX
    ,"AAGIGILTV" : 1 #MLANA
    ,"LLFGFPVYV" : 0 #tax
    ,"ALWGPDPAAA" : 0 #Insuline INS
    ,"ELAAIGILTV" : 1 #MLANA
    ,"ELAGIGALTV" : 1 #MLANA
    ,"ILAKFLHWL" : 1 #TERT
    ,"EAAGIGILTV" : 1 # MLANA
    ,"LLFGYPVAV" : 0 #Tax HTLV
    ,"ALGIGILTV" : 1 #MLANA
    ,"LLLGIGILV" : 0 #BST2
    ,"KLVALGINAV" : 1 #NS3 hepc
    ,"CLGGLLTMV" : 1 #LMP2A
    ,"NLSALGIFST" : 0#IGF2BP2
    ,"MLNIPSINV" :0#pp65
    ,"VAANIVLTV" : 0#SLC30A8
}

In [8]:
a=parse_csv('../NewData/db_pos.csv')
a['label'] = a['epitope'].apply(lambda x: epitope_map[x])
cdr3_label_map = dict(zip(a['cdr3_TRB'].values, a['label'].values))
control = a.query('label==0')
tumor = a.query('label==1')

In [9]:
dc_scaled_feats, dc_scaled_labs = generate_features_labels(tumor['cdr3_TRB'].values,
                                            control['cdr3_TRB'].values,
                                           encoding = 'aaidx', scaling='minmax')

rich_feats, rich_labs = generate_features_labels(tumor['cdr3_TRB'].values,
                                            control['cdr3_TRB'].values,
                                                 encoding='aa_atchley', scaling='minmax')


Getting data
Data device = cpu
Done loading, returning features and labels.
Getting data
Data device = cpu
Done loading, returning features and labels.


In [14]:
_, dc_accs, dc_aucs, dc_f1, dc_curves = test_eval(deepcat_scaled, dc_scaled_feats, dc_scaled_labs,range(12,17), nn.CrossEntropyLoss(), return_curve=True)
_, rich_accs, rich_aucs, rich_f1, rich_curves =  test_eval(richie_minmax, rich_feats, rich_labs,range(12,17), nn.CrossEntropyLoss(), return_curve=True)
plot_roc_comparison(dc_curves, rich_curves, name='RETRAINED MODELS:VDJdb data parsed by Giancarlo',save='vdjdb_giancarlo.jpg')#, keys=range(12,17), save=None)